<a href="https://colab.research.google.com/github/AkankshaB123/leet-code/blob/main/TopNProductsperCategory(Without_Window_Func).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Top N Selling Products For Each Category (Without Window Function)**

Step-by-Step Logic (Using Self-Join or Correlated Query)
1.   t2.sales > t1.sales - beaten by (count) - preferred query always
2.   t1.sales > t2.sales - beats (how many?)



In [2]:
#1.) reg beaten by (count)
# Example:
#   | t1. |  t2.     | beaten by |  RANK
# P1| 100 | 200,300  |   2       |  Top3
# P2| 200 | 100      |   1       |  Top2
# P3| 300 | 0        |   0       |  Top1
#Output: Top 2 - P3,P2

# | t1.product | t1.sales | t2.sales > t1 | beaten_by_count |
# | ---------- | -------- | ------------- | --------------- |
# | P1         | 100      | 200,300       | 2               |
# | P2         | 200      | 300           | 1               |
# | P3         | 300      | 0             | 0               |

In [ ]:
#2.) reg beaten (how many?) - t1.sales > t2.sales
# | t1.product | t1.sales | t2.sales < t1 | beats_count |
# | ---------- | -------- | ------------- | ----------- |
# | P1         | 100      | 0             | 0           |
# | P2         | 200      | 100           | 1           |
# | P3         | 300      | 200,100       | 2           |

# **Using Self-Join**

In [4]:
import sqlite3
import pandas as pd

# ----------------------------
# 1️⃣ Create SQLite in-memory DB
# ----------------------------
conn = sqlite3.connect(":memory:")

# Create table
conn.execute("""
CREATE TABLE txn_data (
    category TEXT,
    product_id TEXT,
    sales INTEGER
)
""")

# ----------------------------
# 2️⃣ Insert sample data
# ----------------------------
data = [
    ('A', 'P1', 100),
    ('A', 'P2', 200),
    ('A', 'P3', 300),
    ('A', 'P4', 150),
    ('B', 'P5', 50),
    ('B', 'P6', 75),
    ('B', 'P7', 125),
]

conn.executemany("INSERT INTO txn_data VALUES (?,?,?)", data)

# ----------------------------
# 3️⃣ Query: beats_count and beaten_by_count
# ----------------------------
query = """
SELECT
    t1.category,
    t1.product_id,
    t1.sales
FROM txn_data t1
LEFT JOIN txn_data t2
  ON t1.category = t2.category
 AND t2.sales > t1.sales
GROUP BY t1.category, t1.product_id, t1.sales
HAVING COUNT(t2.product_id) < 3
ORDER BY t1.category, t1.sales DESC;
"""

# Run the query
df = pd.read_sql_query(query, conn)
df


,category,product_id,sales
0,A,P3,300
1,A,P2,200
2,A,P4,150
3,B,P7,125
4,B,P6,75
5,B,P5,50


# **Using Correlated Sub-Query**

In [5]:
query1 = """SELECT
    t1.category,
    t1.product_id,
    t1.sales
FROM txn_data t1
WHERE (
    SELECT COUNT(*)
    FROM txn_data t2
    WHERE t2.category = t1.category
      AND t2.sales > t1.sales
) < 3
ORDER BY t1.category, t1.sales DESC;
"""

# Run the query
df1 = pd.read_sql_query(query1, conn)
df1

,category,product_id,sales
0,A,P3,300
1,A,P2,200
2,A,P4,150
3,B,P7,125
4,B,P6,75
5,B,P5,50
